In [1]:
# RNN循环神经网络  分类 （时间顺序，图片从上往下读取）
import torch
import torch.nn as nn
import numpy as np
import torchvision.datasets
from torch.utils.data import Dataset,DataLoader
from timm.scheduler.cosine_lr import CosineLRScheduler
import scipy.io as scio
import random
device=torch.device('cuda:0')

In [2]:
class MyDataset(Dataset):
        def __init__(self,data,topk):
            self.data=data
            self.topk=topk
            self.label={
            'T-T':2,
            'P-T':1,
            'P-P':0
            }

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            path_1,label=self.data[idx]
            image=scio.loadmat(path_1)['fc']
            # image=np.nan_to_num(image)
            image=torch.from_numpy(image).float()
            label=torch.tensor(self.label[label])
            return image,label


In [3]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        
        self.dnn = nn.Sequential(
            nn.Linear(64*64,1024),
            nn.Dropout(0.5),
            nn.ELU(),
            nn.Linear(1024,218),
            nn.Dropout(0.5),
            nn.ELU(),
            nn.Linear(218,3),
        )

    def forward(self,x):
        nn1 = x.view(-1,64*64)
        out = self.dnn(nn1)
        return(out)


In [4]:
with open('fc_list.txt','r') as f:
    data=f.readlines()
data=[item.replace('\n','').split('\t') for item in data]
random.seed(2022)
random.shuffle(data)

In [ ]:
result_train_loss,result_loss,result_acc,result_recall=[],[],[],[]
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, roc_auc_score
result_index_matrix=torch.zeros(5,3,3)

k_fold=5
for kk in range(k_fold):
    k_fold_len = int(len(data)//k_fold)
    test_data = data[k_fold_len*kk:k_fold_len*(kk+1)]
    train_data = data[:k_fold_len*kk] + data[k_fold_len*(kk+1):]
    train_dataset=MyDataset(train_data,16)
    test_dataset=MyDataset(test_data,16)
    train_dataloader=DataLoader(train_dataset,batch_size=32,shuffle=True)
    test_dataloader=DataLoader(test_dataset,1)

    model = DNN()
    optimizer=torch.optim.Adam(model.parameters(),lr=1e-4)
    lr_schedule=CosineLRScheduler(optimizer=optimizer,t_initial=10,lr_min=1e-5,warmup_t=5)
    loss_fn= torch.nn.CrossEntropyLoss()
    epochs=60
    loss_fn=loss_fn.to(device)
    model=model.to(device)
    pred_list=[]
    pred_prob_list=[]
    true_list=[]
    index_matrix=torch.zeros(3,3)

    fold_loss,fold_acc,fold_train_loss=1.,0.,1.
    fold_f1,fold_auc=0.,0.
    for epoch in range(epochs):
        model.train()
        train_loss,test_acc,test_loss=.0,.0,.0
        for image,label in train_dataloader:
            image=image.to(device)
            label=label.to(device)
            image.requires_grad = True
            pred=model(image)
            loss=loss_fn(pred,label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
        lr_schedule.step(epoch)
        model.eval()
        for image,label in test_dataloader:
            image=image.to(device)
            label=label.to(device)
            true_list.extend(label.tolist())
            image.requires_grad = True
            pred=model(image)
            pred_list.extend(pred.argmax(dim=1).tolist())
            pred_prob_list.extend(pred.tolist())
            loss=loss_fn(pred,label)
            acc = (pred.argmax(dim=1) == label).float().mean()
        test_acc=accuracy_score(true_list,pred_list)
        test_recall=recall_score(true_list,pred_list,average='micro')
        test_f1=f1_score(true_list,pred_list,average='micro')
        test_auc=roc_auc_score(true_list,pred_prob_list)    
        c_matrix=confusion_matrix(true_list,pred_list)
        for idm in range(3):
            index_matrix[0][idm]=c_matrix[idm][idm]/np.sum(c_matrix[:,idm])
            index_matrix[1][idm]=c_matrix[idm][idm]/np.sum(c_matrix[idm,:])
            index_matrix[2][idm]=(np.sum(c_matrix)-np.sum(c_matrix[:,idm])-np.sum(c_matrix[idm,:])+c_matrix[idm][idm])/(np.sum(c_matrix)-np.sum(c_matrix[idm,:])) 
       
        # print('Epoch: {:2d}  Train Loss: {:.4f}  Test Loss: {:.4f}  Test Acc: {:.4f}'.format(epoch,train_loss/len(train_dataloader),test_loss/len(test_dataloader),test_acc/len(test_dataloader)))
        if test_acc >= fold_acc and (test_f1+test_auc) > (fold_f1+fold_auc):
            fold_acc=test_acc
            result_index_matrix[kk]=index_matrix
            fold_recall=test_recall
    print('Fold{:.0f}:test acc:{:.4f}   test recall:{:.4f}]'.format(kk+1,fold_acc,fold_recall))
    result_train_loss.append(fold_train_loss)
    result_loss.append(fold_loss)
    result_acc.append(fold_acc)
    result_recall.append(fold_recall)
    print(result_index_matrix[kk])
# print('Result: [train loss:{:.4f}  test loss:{:.4f}  test acc:{:.4f}]'.format(np.mean(result_train_loss),np.mean(result_loss),np.mean(result_acc)))
print('Result: [mean:{:.4f}  std:{:.4f}] [mean:{:.4f}  std:{:.4f}]]'.format(np.mean(result_recall),np.std(result_recall),np.mean(result_acc),np.std(result_acc)))